### Burnt Area computation 

This notebook takes two Sentinel-2 products and an area of interest and will generate a GeoTIFF and RGBA as output.

In [ ]:
import snappy
import os
os.environ['PREFIX'] = '/home/eouser/.conda/envs/env_burned_area/'
import sys
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
os.environ['_JAVA_OPTIONS'] = '-Xms1g -Xmx8g'
os.environ['LD_LIBRARY_PATH'] = '.'

import gdal

from shapely.wkt import loads
from shapely.geometry import box, shape, mapping
from shapely.errors import ReadingError
from matplotlib.colors import LinearSegmentedColormap
import shutil
from datetime import datetime
import xml.etree.ElementTree as ET
import requests
import json
import re
from pathlib import Path

from helpers import *

import lxml.etree as etree
from matplotlib.colors import LinearSegmentedColormap


gdal.UseExceptions()

print("Success!")

#### Make the correct settings (area of interest, identifiers, input references, data path and environment variables, including paths)

In [ ]:
aoi_wkt = 'POLYGON((-100.523 25.314,-100.523 25.427,-100.225 25.427,-100.225 25.314,-100.523 25.314))'

input_identifiers = (
    'S2B_MSIL2A_20210305T171109_N0214_R112_T14RLP_20210305T212158',
    'S2B_MSIL2A_20210424T170839_N0300_R112_T14RLP_20210424T211236'
)

data_path = 'input_data'

os.environ['PREFIX'] = '/home/eouser/.conda/envs/env_burned_area'
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')
gpt_path = os.environ['GPT_BIN']
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

sys.path.append('/home/eouser/.conda/envs/env_burned_area/snap/.snap/snap-python')

In [ ]:
namespaces = {
    'xfdu': 'urn:ccsds:schema:xfdu:1',
    'safe': 'http://www.esa.int/safe/sentinel/1.1',
    'gml': 'http://www.opengis.net/gml',
}

metadata = []

for id in input_identifiers:
    with open('{0}/{1}/{1}.SAFE/manifest.safe'.format(data_path, id), 'rb') as manifest:
        manifest_xml = etree.fromstring(manifest.read())

    metadata_elem = manifest_xml.xpath('/xfdu:XFDU/metadataSection', namespaces=namespaces)[0]

    startdate = metadata_elem.xpath('metadataObject/metadataWrap/xmlData/safe:acquisitionPeriod/safe:startTime', namespaces=namespaces)[0].text
    enddate = startdate
    orbitDirection = metadata_elem.xpath('metadataObject/metadataWrap/xmlData/safe:orbitReference/safe:orbitNumber', namespaces=namespaces)[0].get('groundTrackDirection').upper()
    coordinates = metadata_elem.xpath('metadataObject/metadataWrap/xmlData/safe:frameSet/safe:footPrint/gml:coordinates', namespaces=namespaces)[0].text.split(' ')
    wkt = []
    for i in range(len(coordinates) // 2):
        wkt.append("{0} {1}".format(coordinates[2 * i + 1], coordinates[2 * i]))

    metadata.append(
        {
            'identifier': id,
            'startdate': startdate,
            'enddate': enddate,
            'orbitDirection': orbitDirection,
            'wkt': 'POLYGON(({0}))'.format(','.join(wkt))
        }
    )

In [ ]:
for row in metadata:
    row['wkt'] = loads(row['wkt']
)

aoi = loads(aoi_wkt)
(min_lon, min_lat, max_lon, max_lat) = aoi.bounds

print("AOI: ({0},{1}),({2},{3})".format(min_lon, min_lat, max_lon, max_lat))

In [ ]:
print(aoi)
for row in metadata:
    ext = analyse(row, aoi, data_path)
    
for row in metadata:
    print(row)

In [ ]:
composites = []

bands = ['B12', 'B11', 'B8A']

for row in metadata:
    vrt_bands = []
    
    for j, band in enumerate(bands):
        
        vrt_bands.append(get_band_path(row, band))
    
    vrt = '{0}.vrt'.format(row['identifier'])
    ds = gdal.BuildVRT(vrt,
        vrt_bands,
        srcNodata=0,
        xRes=10, 
        yRes=10,
        separate=True
    )
    ds.FlushCache()
    
    tif =  '{0}.tif'.format(row['identifier'])
    
    gdal.Translate(tif,
        vrt,
        projWin=[min_lon, max_lat, max_lon, min_lat],
        projWinSRS='EPSG:4326',
        outputType=gdal.GDT_Byte, 
        scaleParams=[[0, 10000, 0, 255]]
    )
        
        
        
    tif_e =  '{0}_NIR_SWIR_COMPOSITE.tif'.format(row['identifier'])
    
    try:
        composites.append(tif_e)
    except Exception as e:
        print(str(e))
    
    # os.remove(tif)
    # os.remove(vrt)

In [ ]:
pre_processed = []

resample = dict()
resample['referenceBandName'] = 'B2'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

subset = dict()
subset['geoRegion'] = box(*aoi.bounds).wkt
subset['copyMetadata'] = 'true'

bands = '''<targetBands>
    <targetBand>
      <name>NDWI</name>
      <type>float32</type>
      <expression>(B3 - B8) / (B3 + B8)</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>NBR</name>
      <type>float32</type>
      <expression>(B8 - B12) / (B8 + B12)</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>valid_pixels</name>
      <type>float32</type>
      <expression>scl_vegetation or scl_not_vegetated ? 1 : 0</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    </targetBands>'''

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 



for row in metadata:
    print(os.path.join(row['local_path'], row['identifier'] + '.SAFE', 'MTD_MSIL2A.xml'))
    
    read = dict()
    read['file'] = os.path.join(row['local_path'], row['identifier'] + '.SAFE', 'MTD_MSIL2A.xml') #, 'manifest.safe')
    #read['formatName'] = 'SENTINEL-2-MSI-MultiRes-UTM52N'
    
    write = dict()
    write['file'] = 'pre_{}'.format(row['identifier'])

    row['pre_proc'] = 'pre_{}'.format(row['identifier'])
    
    
    pre_processed.append('pre_{}'.format(row['identifier']))

    print("*******")
    print("IDENTIFIER = {0}".format(row['identifier']))
    print("READ = {0}".format(read))
    print("WRITE = {0}".format(write))
    print("*******")
    
    
    pre_processing(Read=read, 
        Resample=resample, 
        Reproject=reproject, 
        Subset=subset,
        BandMaths=band_maths,
        Write=write
    )

    print(metadata)

In [ ]:
master_path = "{0}.dim".format(min((r['pre_proc'] for r in metadata)))
slave_path = "{0}.dim".format(max((r['pre_proc'] for r in metadata)))

#master_path='pre_S2B_MSIL2A_20201202T184739_N0214_R070_T11SKB_20201202T205847.dim'
#slave_path='pre_S2A_MSIL2A_20201207T184751_N0214_R070_T11SKB_20201207T205922.dim'

print(master_path)
print(slave_path)

In [ ]:
mygraph = GraphProcessor(os.path.join(os.environ['PREFIX'], 'snap/bin/gpt'))
operator = 'Read'

node_id = 'Read_M'

source_node_id = ''

parameters = get_operator_default_parameters(operator)
     
parameters['file'] = master_path 
    
mygraph.add_node(node_id, operator, parameters, source_node_id)

operator = 'Read'

node_id = 'Read_S'

source_node_id = ''

parameters = get_operator_default_parameters(operator)
     
parameters['file'] = slave_path   
    
mygraph.add_node(node_id, operator, parameters, source_node_id)

operator = 'Collocate'

parameters = get_operator_default_parameters(operator)

parameters['masterComponentPattern'] = 'PRE_FIRE_${ORIGINAL_NAME}'
parameters['slaveComponentPattern'] = 'POST_FIRE_${ORIGINAL_NAME}'

source_node_id = dict()

source_node_id['master'] = 'Read_M'

source_node_id['slave'] = 'Read_S'


node_id = 'Collocate'

mygraph.add_node(operator, operator,  parameters, source_node_id)

operator = 'Write'

node_id = 'Write'

source_node_id = 'Collocate'



parameters = get_operator_default_parameters(operator)

parameters['file'] = 'collocated'
parameters['formatName'] = 'BEAM-DIMAP'

mygraph.add_node(node_id, operator, parameters, source_node_id)

In [ ]:
mygraph.save_graph(filename='graph.xml')
mygraph.run()

In [ ]:
output_name = 'burned_area_{0}_{1}'.format(
    datetime.strptime(min(r['enddate'] for r in metadata)[:19], '%Y-%m-%dT%H:%M:%S').strftime('%Y%m%d_%H%M%S'),
    datetime.strptime(max(r['enddate'] for r in metadata)[:19], '%Y-%m-%dT%H:%M:%S').strftime('%Y%m%d_%H%M%S')
)

In [ ]:
collocated_input = 'collocated.dim'

read = dict()
read['file'] = collocated_input

bands = '''<targetBands>
    <targetBand>
      <name>dNBR</name>
      <type>float32</type>
      <expression>(PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1 and ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27) ? PRE_FIRE_NBR - POST_FIRE_NBR : -999</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>RBR</name>
      <type>float32</type>
      <expression>(PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1 and ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27) ? ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) : -999</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>valid_pixels</name>
      <type>float32</type>
      <expression>PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    </targetBands>'''

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 

write = dict()
write['file'] = output_name
write['formatName'] = 'GeoTIFF'

burned_area(
    Read=read, 
    BandMaths=band_maths,
    Write=write
)

In [ ]:
create_rgba(
    output_name + '.tif',
    output_name + '.rgb.tif'
)

In [ ]:
band_names = ['dNBR',
              'RBR',
             'valid_pixels']

expressions = ['PRE_FIRE_NDWI >= 0.0 ? 0 : ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27 ? PRE_FIRE_NBR - POST_FIRE_NBR : 0',
               'PRE_FIRE_NDWI >= 0.0 ? 0 : ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27 ? ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) : 0',
              'PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1']

ds_temp = gdal.Open(output_name + '.tif',  gdal.OF_UPDATE)

for band_index in range(ds_temp.RasterCount):

    band_metadata = dict()
    band_metadata['BAND_EXPRESSION'] = expressions[band_index]

    src_band = ds_temp.GetRasterBand(band_index+1)
    src_band.SetMetadata(band_metadata)
    src_band.SetDescription(band_names[band_index])  

ds_temp.FlushCache()

END